In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import pickle

# -------- Data Loading Function --------
def load_data(folder_path):
    all_data = []
    for file in sorted(os.listdir(folder_path)):
        if file.endswith('.csv'):
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path)
            if '5 Minutes' in df.columns and 'Flow (Veh/5 Minutes)' in df.columns and 'Speed (mph)' in df.columns:
                # Convert timestamp column to datetime
                df['5 Minutes'] = pd.to_datetime(df['5 Minutes'], errors='coerce')
                # Set timestamp as the index
                df = df.set_index('5 Minutes')
                # Append the required columns
                all_data.append(df[['Flow (Veh/5 Minutes)', 'Speed (mph)']])
    
    # Concatenate all dataframes
    combined_data = pd.concat(all_data)
    
    # Sort by the timestamp index
    combined_data = combined_data.sort_index()
    
    # Drop any rows with missing values
    combined_data = combined_data.dropna()
    
    return combined_data

# Load data for each node
folder_402214 = 'Data/402214'
folder_402510 = 'Data/402510'
folder_402835 = 'Data/402835'
folder_414025 = 'Data/414025'

data_402214 = load_data(folder_402214)
data_402510 = load_data(folder_402510)
data_402835 = load_data(folder_402835)
data_414025 = load_data(folder_414025)

# Find the common date range
common_start = max(data_402214.index.min(), data_402510.index.min(), data_402835.index.min(), data_414025.index.min())
common_end = min(data_402214.index.max(), data_402510.index.max(), data_402835.index.max(), data_414025.index.max())

# Create a common date range with 5-minute intervals
common_index = pd.date_range(start=common_start, end=common_end, freq='5T')

# Reindex and interpolate data for each node
data_402214 = data_402214.reindex(common_index).interpolate()
data_402510 = data_402510.reindex(common_index).interpolate()
data_402835 = data_402835.reindex(common_index).interpolate()
data_414025 = data_414025.reindex(common_index).interpolate()

# Stack the data for all nodes
node_data = np.stack([
    data_402214[['Flow (Veh/5 Minutes)', 'Speed (mph)']].values,
    data_402510[['Flow (Veh/5 Minutes)', 'Speed (mph)']].values,
    data_402835[['Flow (Veh/5 Minutes)', 'Speed (mph)']].values,
    data_414025[['Flow (Veh/5 Minutes)', 'Speed (mph)']].values
], axis=0)

# Normalize the data using MinMaxScaler
scalers = [MinMaxScaler() for _ in range(node_data.shape[0])]
for i in range(node_data.shape[0]):
    # Apply MinMaxScaler to each node's data
    node_data[i] = scalers[i].fit_transform(node_data[i])

# Transpose the data to shape (num_timesteps, num_nodes, num_features)
node_data = np.transpose(node_data, (1, 0, 2))  # Shape: (num_timesteps, num_nodes, num_features)

# -------- 2. Create Adjacency Matrix --------
# Load the adjacency matrix (connectivity between nodes)
adj_matrix = pd.read_csv('Data/adj.csv', header=None).values

# Normalize the adjacency matrix
def normalize_adjacency_matrix(adj):
    D = np.diag(np.sum(adj, axis=1))
    D_inv = np.linalg.inv(D)
    adj_normalized = np.dot(D_inv, adj)
    return adj_normalized

adj_normalized = normalize_adjacency_matrix(adj_matrix)
# Ensure adj_normalized is a PyTorch tensor with correct shape
adj_normalized = adj_normalized[:4, :4]  # Keep only the first 4x4 submatrix
adj_normalized = torch.tensor(adj_normalized, dtype=torch.float32)

# -------- 3. Prepare the Data --------
# Split the data into training, validation, and test sets
train_size = int(0.7 * len(node_data))
val_size = int(0.15 * len(node_data))
test_size = len(node_data) - train_size - val_size

train_data, val_data, test_data = np.split(node_data, [train_size, train_size + val_size])

# Create DataLoader for train, val, and test data
def create_dataloader(data, batch_size, input_window=10, output_window=5):
    X, Y = [], []
    for i in range(len(data) - input_window - output_window + 1):
        X.append(data[i:i+input_window])
        # Predict only the first feature, which is 'Flow (Veh/5 Minutes)'
        Y.append(data[i+input_window:i+input_window+output_window, :, 0])  # Change here
    X = torch.tensor(np.array(X), dtype=torch.float32)
    Y = torch.tensor(np.array(Y), dtype=torch.float32)
    dataset = TensorDataset(X, Y)
    return DataLoader(dataset, batch_size=batch_size, shuffle=False)

batch_size = 32
train_loader = create_dataloader(train_data, batch_size)
val_loader = create_dataloader(val_data, batch_size)
test_loader = create_dataloader(test_data, batch_size)

# -------- 4. Build the Model --------
# GCN Layer
class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GCNLayer, self).__init__()
        self.fc = nn.Linear(in_features, out_features)

    def forward(self, x, adj):
        out = torch.einsum('bni,nj->bni', x, adj)
        out = self.fc(out)
        return torch.relu(out)

# GCN Model
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.gcn1 = GCNLayer(input_dim, hidden_dim)
        self.gcn2 = GCNLayer(hidden_dim, output_dim)

    def forward(self, x, adj):
        x = self.gcn1(x, adj)
        x = self.gcn2(x, adj)
        return x

# Combined GNN + LSTM Model
class GNN_LSTM_Model(nn.Module):
    def __init__(self, num_nodes, input_dim, gnn_hidden_dim, lstm_hidden_dim, output_dim, output_window):
        super(GNN_LSTM_Model, self).__init__()
        self.gcn = GCN(input_dim, gnn_hidden_dim, gnn_hidden_dim)
        self.lstm = nn.LSTM(gnn_hidden_dim * num_nodes, lstm_hidden_dim, batch_first=True)
        self.fc = nn.Linear(lstm_hidden_dim, num_nodes * output_dim * output_window)
        self.num_nodes = num_nodes
        self.output_dim = output_dim
        self.output_window = output_window

    def forward(self, x, adj):
        batch_size, input_window, num_nodes, input_dim = x.shape
        
        gnn_out = []
        for t in range(input_window):
            gnn_output = self.gcn(x[:, t, :, :], adj)
            gnn_out.append(gnn_output)
        
        # Stack GNN outputs
        gnn_out = torch.stack(gnn_out, dim=1)  # (batch_size, input_window, num_nodes, gnn_hidden_dim)
        
        # Reshape for LSTM input
        lstm_in = gnn_out.view(batch_size, input_window, -1)
        
        # Pass through LSTM
        lstm_out, _ = self.lstm(lstm_in)  # (batch_size, input_window, lstm_hidden_dim)
        
        # Use only the last output from LSTM
        fc_in = lstm_out[:, -1, :]
        
        # Pass through fully connected layer
        fc_out = self.fc(fc_in)  # (batch_size, num_nodes * output_dim * output_window)
        
        # Reshape output to (batch_size, output_window, num_nodes, output_dim)
        output = fc_out.view(batch_size, self.output_window, self.num_nodes, self.output_dim)
        
        return output

# Model instantiation
num_nodes = 4
input_dim = 2  # Number of features: 'Flow' and 'Speed'
gnn_hidden_dim = 128
lstm_hidden_dim = 256
output_dim = 1  # We're predicting only one feature: 'Flow (Veh/5 Minutes)'
output_window = 5

model = GNN_LSTM_Model(num_nodes, input_dim, gnn_hidden_dim, lstm_hidden_dim, output_dim, output_window)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# -------- 5. Training the Model --------
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for X_batch, Y_batch in tqdm(train_loader):
        optimizer.zero_grad()
        output = model(X_batch, adj_normalized)
        # Reshape Y_batch for loss calculation
        Y_batch = Y_batch.view(Y_batch.shape[0], output_window, num_nodes, output_dim)  # (batch_size, output_window, num_nodes, output_dim)
        loss = criterion(output, Y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader):.4f}')

# -------- 6. Validation --------
model.eval()
val_loss = 0
with torch.no_grad():
    for X_batch, Y_batch in val_loader:
        output = model(X_batch, adj_normalized)
        Y_batch = Y_batch.view(Y_batch.shape[0], output_window, num_nodes, output_dim)
        loss = criterion(output, Y_batch)
        val_loss += loss.item()

print(f'Validation Loss: {val_loss/len(val_loader):.4f}')

# -------- 7. Testing --------
model.eval()
test_loss = 0
predictions = []
true_values = []
with torch.no_grad():
    for X_batch, Y_batch in test_loader:
        output = model(X_batch, adj_normalized)
        predictions.append(output)
        true_values.append(Y_batch)

# Convert predictions and true values to numpy arrays
predictions = torch.cat(predictions, dim=0).numpy()
true_values = torch.cat(true_values, dim=0).numpy()

# Calculate test loss
test_loss = criterion(torch.tensor(predictions), torch.tensor(true_values))
print(f'Test Loss: {test_loss.item():.4f}')

# -------- Save Model and Scalers --------
torch.save(model.state_dict(), 'gnn_lstm_model.pth')

with open('scalers.pkl', 'wb') as f:
    pickle.dump(scalers, f)


100%|██████████| 529/529 [00:08<00:00, 62.19it/s]


Epoch 1/50, Loss: 0.0136


100%|██████████| 529/529 [00:08<00:00, 62.40it/s]


Epoch 2/50, Loss: 0.0054


100%|██████████| 529/529 [00:08<00:00, 63.10it/s]


...

Epoch 49/50, Loss: 0.0022


100%|██████████| 529/529 [00:08<00:00, 60.66it/s]


Epoch 50/50, Loss: 0.0022
Validation Loss: 0.0021


/Users/hiyabhatt/miniforge3/envs/mlp/lib/python3.8/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([3616, 5, 4])) that is different to the input size (torch.Size([3616, 5, 4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (4) must match the size of tensor b (5) at non-singleton dimension 2